# Introduction to `py_trees` library

Behaviour trees are a decision making engine often used in the gaming and robotics industries. [py_trees](https://github.com/splintered-reality/py_trees) is a python implementation of a Behavior Trees engine that you can use in your projects.

## Documentation

All the py_trees documnetation can be found [here](https://py-trees.readthedocs.io/en/devel/index.html).

As a summary, `py_trees` combine the following elements:

* **Behaviors**: Behaviours are the leaves of the behaviour tree. This is where actions and conditions are done. Some generic behaviors are already implemented in the `py_trees` library. However, most of the particular behaviors must be implemented by inheriting from the `Behavior` class.
* **Composites**: Control flow nodes. Three basic composite nodes are used: *Sequence*, *Selector*, and *Parallel*.
* **Decorators**: Nodes with a single childres that modify their underlying child behaviour. Several decorators are available in `py_trees`. Some of the most popular are: *Count*, *Inverter*, *OneShot*, *Repeat*, *Timeout*, ...

## Example

Next, a super basic example is provided.
We want to implement a behavior tree that checks if we are busy or not. If we are not busy then we can take a beer for a limited time. The whole process can be repeated as many times as necessary.

This task is implemented following this behavior tree.

![](./life_1.png)

It shows a behavior tree with one condition (i.e., *Busy*) and one action (i.e., *Have a Beer*), both implementd as custom behaviors, a couple of decorators (i.e., *Inverter* and *Timeout*), and one composite (i.e., *Sequence*).

### Define the behaviors

The two custom behaviors are implemented as follows:

* The *Busy* condition may return only `SUCCESS` or `FAILURE`. In this naive implementation this is selected randomly.
* The *Have a Beer* action could return `SUCCESS`, `FAILURE`, or `RUNNING` but for simplicity it will always return `RUNNING`.

Because both Behaviors inherit from the `py_trees.behaviour.Behaviour` class, several methods must be implemented. Check the documentation to understand why is each function used for.

In [1]:
import py_trees
import random

# Example of a behavior
class Busy(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(Busy, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [Busy::setup()]" % self.name)
        
    def initialise(self):
        self.logger.debug("  %s [Busy::initialise()]" % self.name)
        
    def update(self):
        self.logger.debug("  %s [Busy::update()]" % self.name)
        ready_to_make_a_decision = random.choice([True, False])
        decision = random.choice([True, False])
        if not ready_to_make_a_decision:
            self.logger.debug("  %s [Busy::update() RUNNING]" % self.name)
            return py_trees.common.Status.RUNNING
        elif decision:
            self.feedback_message = "We are not bar!"
            self.logger.debug("  %s [Busy::update() SUCCESS]" % self.name)
            return py_trees.common.Status.SUCCESS
        else:
            self.feedback_message = "Uh oh"
            self.logger.debug("  %s [Busy::update() FAILURE]" % self.name)
            return py_trees.common.Status.FAILURE

    def terminate(self, new_status):
        self.logger.debug("  %s [Busy::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))


class HaveABeer(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(HaveABeer, self).__init__(name)

    def setup(self):
        self.logger.debug("  %s [HaveABeer::setup()]" % self.name)
        
    def initialise(self):
        self.logger.debug("  %s [HaveABeer::initialise()]" % self.name)
        
    def update(self):
        self.logger.debug("  %s [HaveABeer::update()]" % self.name)
        self.logger.debug("  %s [HaveABeer::update() RUNNING]" % self.name)
        return py_trees.common.Status.RUNNING
        
    def terminate(self, new_status):
        self.logger.debug("  %s [HaveABeer::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))


### Create the behavior tree

Now, we are going to connect the previously generated behaviors using decorators and composite structures to build the ddesired tree. Check the available documentation for more information.

In [2]:
import py_trees.decorators
import py_trees.display

def create_tree():
    busy = Busy(name="Busy?")
    
    inverter = py_trees.decorators.Inverter(
        name="Inverter",
        child=busy
    )
    
    have_beer = HaveABeer(name="Have a Beer!")
    
    timeout = py_trees.decorators.Timeout(
        name="Timeout",
        duration=3,
        child=have_beer
    )

    root = py_trees.composites.Sequence(name="Life", memory=True)
    root.add_children([inverter, timeout])
    
    return root

root = create_tree()
py_trees.display.render_dot_tree(root) # generates a figure for the 'root' tree.

Writing /home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.dot
Writing /home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.png
Writing /home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.svg


{'dot': '/home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.dot',
 'png': '/home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.png',
 'svg': '/home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.svg'}

## Run the behavior Tree

To run the behavior, first we need to call the `setup()` method for all the behaviors in the tree. Then we can call the `tick_once()` method to play it.

In [3]:
import time
py_trees.logging.level = py_trees.logging.Level.DEBUG

def run(it=10):
    root = create_tree()

    try:
        print("Call setup for all tree children")
        root.setup_with_descendants() 
        print("Setup done!\n\n")
        py_trees.display.ascii_tree(root)
        
        for _ in range(it):
            root.tick_once()
            time.sleep(1)
    except KeyboardInterrupt:
        pass


In [4]:
# Run the bBehavior Tree for 10 steps
run()

Call setup for all tree children
[DEBUG] Busy?                :   Busy? [Busy::setup()]
[DEBUG] Have a Beer!         :   Have a Beer! [HaveABeer::setup()]
Setup done!


[DEBUG] Life                 : Sequence.tick()
[DEBUG] Inverter             : Inverter.tick()
[DEBUG] Busy?                : Busy.tick()
[DEBUG] Busy?                :   Busy? [Busy::initialise()]
[DEBUG] Busy?                :   Busy? [Busy::update()]
[DEBUG] Busy?                :   Busy? [Busy::update() FAILURE]
[DEBUG] Busy?                : Busy.stop(Status.INVALID->Status.FAILURE)
[DEBUG] Busy?                :   Busy? [Busy::terminate().terminate()][Status.INVALID->Status.FAILURE]
[DEBUG] Inverter             : Inverter.stop(Status.SUCCESS)
[DEBUG] Timeout              : Timeout.tick()
[DEBUG] Have a Beer!         : HaveABeer.tick()
[DEBUG] Have a Beer!         :   Have a Beer! [HaveABeer::initialise()]
[DEBUG] Have a Beer!         :   Have a Beer! [HaveABeer::update()]
[DEBUG] Have a Beer!         :   Have a Bee

## Introduction to the Blackboard

Blackboards are a common mechanism for sharing data between behaviours in the tree.
In `py_trees`, nodes have to register which vars they want to `READ` or `WRITE`.

```python
self.blackboard = self.attach_blackboard_client(name=self.name)
self.blackboard.register_key("var_name", access=py_trees.common.Access.WRITE)
self.blackboard.register_key("var_name", access=py_trees.common.Access.READ)
```

Lets modify the previous behavior tree to count how many beers we have had, and just have a beer if we have had less than 3 beers.

In [5]:

# Modify the HaveABeer behavior to store in tha black Board how many beers it have had.
class HaveABeer(py_trees.behaviour.Behaviour):
    def __init__(self, name):
        super(HaveABeer, self).__init__(name)
        self.blackboard = self.attach_blackboard_client(name=self.name)
        self.blackboard.register_key("n_beers", access=py_trees.common.Access.WRITE)
        self.blackboard.register_key("n_beers", access=py_trees.common.Access.READ)
        self.beers = None
        
    def setup(self):
        self.logger.debug("  %s [HaveABeer::setup()]" % self.name)
        self.blackboard.n_beers = 0
        
    def initialise(self):
        self.logger.debug("  %s [HaveABeer::initialise()]" % self.name)
        self.beers = self.blackboard.n_beers
        print("Beers already had ", self.beers)

    def update(self):
        self.logger.debug("  %s [HaveABeer::update()]" % self.name)
        return py_trees.common.Status.RUNNING
        
    def terminate(self, new_status):
        self.logger.debug("  %s [HaveABeer::terminate().terminate()][%s->%s]" % (self.name, self.status, new_status))
        self.blackboard.n_beers = self.beers + 1

Now, lets modify the tree structure to check how many beers it have had, according to the value in the Black Board.

In [6]:
import operator 

def create_tree():

    busy = Busy(name="Busy?")
    inverter = py_trees.decorators.Inverter(
        name="Inverter",
        child=busy
    )

    # Special py_trees behavior
    n_beers_lt_2 = py_trees.behaviours.CheckBlackboardVariableValue(
        name="n_beers_lt_2",
        check=py_trees.common.ComparisonExpression(
            variable="n_beers",
            value=2,
            operator=operator.lt
        )
    )

    have_beer = HaveABeer(name="Have a Beer!")
    timeout = py_trees.decorators.Timeout(
        name="Timeout",
        duration=3,
        child=have_beer
    )

    check_n_beers = py_trees.composites.Sequence(name="check_n_beers", memory=True)
    check_n_beers.add_children([n_beers_lt_2, timeout])
    
    root = py_trees.composites.Sequence(name="Life", memory=True)    
    root.add_children([inverter, check_n_beers])
    
    return root


root = create_tree()
py_trees.display.render_dot_tree(root)

Writing /home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.dot
Writing /home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.png
Writing /home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.svg


{'dot': '/home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.dot',
 'png': '/home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.png',
 'svg': '/home/sazid/catkin_ws/src/pick_up_objects_task/notebooks/life.svg'}

In [7]:
# Run the Behavior Tree for 30 steps
run(it=30)

Call setup for all tree children
[DEBUG] Busy?                :   Busy? [Busy::setup()]
[DEBUG] Have a Beer!         :   Have a Beer! [HaveABeer::setup()]
Setup done!


[DEBUG] Life                 : Sequence.tick()
[DEBUG] Inverter             : Inverter.tick()
[DEBUG] Busy?                : Busy.tick()
[DEBUG] Busy?                :   Busy? [Busy::initialise()]
[DEBUG] Busy?                :   Busy? [Busy::update()]
[DEBUG] Busy?                :   Busy? [Busy::update() RUNNING]
[DEBUG] Life                 : Sequence.tick()
[DEBUG] Inverter             : Inverter.tick()
[DEBUG] Busy?                : Busy.tick()
[DEBUG] Busy?                :   Busy? [Busy::update()]
[DEBUG] Busy?                :   Busy? [Busy::update() RUNNING]
[DEBUG] Life                 : Sequence.tick()
[DEBUG] Inverter             : Inverter.tick()
[DEBUG] Busy?                : Busy.tick()
[DEBUG] Busy?                :   Busy? [Busy::update()]
[DEBUG] Busy?                :   Busy? [Busy::update() RUNNING]
